# Deepface Face Recognition Testing

## Import relevant libraries

In [1]:
# !pip install deepface
from deepface import DeepFace

import matplotlib.pyplot as plt
import pandas as pd                 
import regex as re

# directory cleaning
import os
from os import listdir
from os.path import isfile, join

# change plot output to white
params = {"ytick.color" : "w",
          "xtick.color" : "w",
          "axes.labelcolor" : "w",
          "axes.edgecolor" : "w"}
plt.rcParams.update(params)

# result_df = pd.read_csv('./result/result_debface.csv')
# benchmark_df = pd.read_csv('./data/LFW-csv/pairs.csv')

In [ ]:
# result_df

In [ ]:
# benchmark_df.head()

### Parameter settings:

In [2]:
# select a DATASET from data folder:
# LFW, LFW-original
DATASET = "LFW"

# select a dataset from additional-data folder:
# tzuyu, kpop
DATASET_TEST = "tzuyu"

# select a model to run:
# models = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib", "SFace"]
MODEL = "Facenet"

#### Reset directory and test instances 

In [8]:
import os

# Test additional data directory:
# dir_path = f"./additional-data/{DATASET_TEST}"

# Test dataset directory:
dir_path = f"./data/{DATASET}"

# list all files in the directory
all_dir_path = os.listdir(dir_path)

# Removes previously stored representations_FACE.pkl, as adding new instances requires to re-test the model
## For LFW dataset
for item in all_dir_path:
    item_path = f"./data/{DATASET}/{item}"
    sub_files = os.listdir(item_path) 
    for sub_item in sub_files:
        if sub_item.endswith(".pkl"):
            os.remove(os.path.join(item_path, sub_item))

## For random data dataset
# for item in all_dir_path:
#     if item.endswith(".pkl"):
#         os.remove(os.path.join(dir_path, item))

## Function to test Deepface Face recognition 

### Find accuracy of specific person

In [88]:
def deepface_get_accuracy(model_used, dir_path):
    # Recoginize all images in the dataset and verify them
    # calculate accuracy by dividing the number true image by the total number of images recognized

    # store the results of each verification:
    tp, fp, tn, fn = 0, 0, 0, 0
    total_images = os.listdir(dir_path)
    num_images = len(total_images)

    # Skip if folder only contains 1 image
    if num_images < 1:
        return None

    else:
        try:
            # select the first image as image to recognise 
            image_truth = f"{dir_path}/{total_images[0]}"
            
            # set enforce_detection to False for full body images  
            df = pd.DataFrame(DeepFace.find(img_path = image_truth, 
                                            db_path = dir_path, 
                                            model_name = model_used, 
                                            enforce_detection=False))
            
            # declare truth images, noise, image_name
            truth_images = []
            noise = []
            match = re.search(f"_\d.*\.jpg", str(total_images[0]))
            NAME = total_images[0].replace(match.group(0), "")

            # find total ground truth images
            for image in total_images[1:]:
                if re.search(f"{NAME}_\d.*\.jpg", str(image)):                                          
                    truth_images.append(image) 
                else:
                    noise.append(image)
            
            # loop through the images that were recognised 
            for image in df['identity'][1:]:
                if re.search(f".*\/{NAME}_\d.*\.jpg", str(image)):
                    tp += 1
                else:
                    fp += 1

            tn = len(noise) - fp
            fn = len(truth_images) - tp

            # calculate confusion matrix accuracy
            cm_acc = round( (tp + tn)/(tp + tn + fp + fn), 2) * 100
            
        except Exception as e:
            print(e)




        return {"Model": model_used, "Dataset": DATASET, "CM_Accuracy": cm_acc, "Total Images": num_images-1}, \
                {"tp": tp, "tn":tn, "fp":fp, "fn":fn, "truth":truth_images, "noise":noise}


#### Find accuracy for entire dataset

In [85]:
def deepface_dataset_fr(model_used, dir_path):
    avg_truth_pos = 0
    avg_truth_neg = 0
    avg_false_pos = 0
    avg_false_neg = 0
    total_imgs = 0
    
    # load the people from the dataset
    people_images = os.listdir(dir_path)

    # loop through each people and execute the deepface face recognition test
    for people in people_images:
        people_img_path = f"{dir_path}/{people}"
        
        deepface_res, cm_attr = deepface_get_accuracy(model_used, people_img_path)

        if deepface_res and cm_attr == None:
            pass
        else:
            avg_truth_pos += cm_attr["tp"]
            avg_truth_neg += cm_attr["tn"]
            avg_false_pos += cm_attr["fp"]
            avg_false_neg += cm_attr["fn"]
            total_imgs += deepface_res["Total Images"]

    # calculate confusion matrix:
    # accuracy:
    cm_acc = round((avg_truth_pos + avg_truth_neg) / (avg_truth_pos + avg_truth_neg + avg_false_pos + avg_false_neg), 2) * 100
    # precision:
    cm_pre = round( (avg_truth_pos) / (avg_truth_pos + avg_false_pos), 2) * 100
    # recall: 
    cm_rec = round( (avg_truth_pos) / (avg_truth_pos + avg_false_neg), 2) * 100

    return {"Model": model_used, "Dataset": DATASET, "CM_Accuracy": cm_acc, "Precision":cm_pre, "Recall":cm_rec, "Total Images": total_imgs},\
            {"TP": avg_truth_pos, "TN":avg_truth_neg, "FP":avg_false_pos, "FN":avg_false_neg}
    

### Execution of Deepface testing:

In [76]:
# Run Deepface face recognition testing:
deepface_res, cm_attr = deepface_get_accuracy(MODEL, dir_path)

1/1 [==============================] - 0s 35ms/step
Representations stored in  ./additional-data/tzuyu / representations_facenet.pkl  file. Please delete this file when you add new identities in your database.
1/1 [==============================] - 0s 39ms/step
find function lasts  0.8691086769104004  seconds


In [89]:
# Run Deepface face recognition on entire dataset:
deepface_res_dataset, cm_attr = deepface_dataset_fr(MODEL, dir_path)

There are  1  representations found in  representations_facenet.pkl
1/1 [==============================] - 0s 36ms/step
find function lasts  0.11499834060668945  seconds
1/1 [==============================] - 0s 36ms/step
Representations stored in  ./data/LFW/Aaron_Guiel / representations_facenet.pkl  file. Please delete this file when you add new identities in your database.
1/1 [==============================] - 0s 35ms/step
find function lasts  0.1995527744293213  seconds
division by zero


UnboundLocalError: local variable 'cm_acc' referenced before assignment

### Results from Deepface testing:

In [77]:
# print(f'Accuracy of {model_used} is {acc}% out of {len(result)} images')
print(deepface_res)
print(cm_attr)

{'Model': 'Facenet', 'Dataset': 'LFW', 'CM_Accuracy': 75.0, 'Total Images': 4}
{'tp': 3, 'tn': 0, 'fp': 0, 'fn': 1, 'truth': ['tzuyu_0002.jpg', 'tzuyu_0003.jpg', 'tzuyu_0004.jpg', 'tzuyu_0008_wear.jpg'], 'noise': []}


### Writing results to CSV 

In [ ]:
result_df_new = pd.DataFrame([deepface_res])

frames = [result_df, result_df_new]
result_df = pd.concat(frames)

result_df = result_df.reset_index()
result_df.drop(columns=['index'], inplace=True)
result_df

In [ ]:
# Write results to csv file
result_df.to_csv("./result/result_debface_fr.csv", index=False)